In [1]:
!pip install hezar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=38a69a3df6352635ec4cbb77661b48bbd0cc4a7934f61e3e6539c7722652105c
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime
  Attempting uninstall: pyarrow
    Found existing i

In [2]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.7 MB/s eta 0:00:00


In [3]:
!pip install convert_numbers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.6/319.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for convert_numbers: filename=convert_numbers-0.4-py3-none-any.whl size=3804 sha256=0a63cde9199f70220ab72d8f80641a5588bce948f6d1ecaa92bd505d7b518ffa
  Stored in directory: /root/.cache/pip/wheels/4b/1c/41/434286e272577c04f246119d6be4f2d23d017cb677b9bbe582
Successfully built convert_numbers


In [4]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00


In [32]:
import easyocr
import cv2
import os
import numpy as np
from PIL import Image
import convert_numbers
from hezar.models import Model
from PIL import Image, ImageEnhance

def enhance_image(input_path, output_path, contrast_factor=1.5, sharpness_factor=2.0):
    # Open the image file
    image = Image.open(input_path)
    # Enhance contrast
    contrast = ImageEnhance.Contrast(image)
    image = contrast.enhance(contrast_factor)

    # Enhance sharpness
    sharpness = ImageEnhance.Sharpness(image)
    image = sharpness.enhance(sharpness_factor)
    # Save the enhanced image
    image.save(output_path)

def ocr_and_save_segments(image_path, output_folder, language='fa'):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    enhance_image(image_path, "tmp.jpg")

    # Load the image
    image = cv2.imread("tmp.jpg")

    # Initialize the EasyOCR reader with the specified language
    reader = easyocr.Reader([language])

    # Perform OCR on the image
    result = reader.readtext(image)

    # Process each detected segment
    for i, detection in enumerate(result):
        points = detection[0]
        box = [[int(x), int(y)] for (x, y) in points]
        box = np.array(box)

        # Extract the region of interest (ROI) from the image
        x, y, w, h = cv2.boundingRect(box)
        segment = image[y:y+h, x:x+w]

        # Save the segment to the output folder
        segment_filename = f'segment_{i + 1}.png'
        segment_path = os.path.join(output_folder, segment_filename)
        cv2.imwrite(segment_path, segment)

def ocr_with_hezar(image_path, model):
    # Load the image
    image = Image.open(image_path)

    # OCR with CRNN
    texts = model.predict(image)
    return texts[0]['text']

def check(target_text, all_texts):
  for text in all_texts:
      if target_text == text:
          return 1
  return 0

def func(file_name, entered_text):
    # Example usage for Farsi (Persian)
    image_path = file_name
    output_folder = '/content/output_segments' +  file_name
    ocr_and_save_segments(image_path, output_folder, language='fa')

    # Example usage for Farsi (Persian)
    image_folder = output_folder

    # Process each segmented image in the folder
    for filename in os.listdir(image_folder):
        if filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)

    # لیستی برای ذخیره تمام متن‌ها
    all_texts = []

    # OCR with TrOCR
    model1 = Model.load("hezarai/trocr-base-fa-v2")

    # OCR with CRNN
    model2 = Model.load("hezarai/crnn-fa-printed-96-long")

    # Process each segmented image in the folder
    for filename in os.listdir(image_folder):
        if filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)

            # Run OCR with hezar on the segmented image
            text = ocr_with_hezar(image_path, model1)
            all_texts.append(text)

            text = ocr_with_hezar(image_path, model2)
            all_texts.append(text)

    # print(all_texts)

    if(check(entered_text['نام:'], all_texts)):
        print("نام معتبر است")
    else:
        print("نام اشتباه وارد شده!")

    sw2 = 0

    words = entered_text['نام خانوادگی:'].split()


    if(check(entered_text['نام خانوادگی:'], all_texts)):
        sw2 = 1
    elif(len(words)>1):
        sw2 = 1
        for i in range(len(words)):
            if not(check(words[i], all_texts)):
                sw = 0
                break

    if(sw2):
        print("نام خانوادگی معتبر است")
    else:
        print("نام خانوادگی اشتباه وارد شده!")

    if(check(entered_text['نام مادر:'], all_texts)):
        print("نام مادر معتبر است")
    else:
        print("نام مادر اشتباه وارد شده!")
    if(check(entered_text['نام پدر:'], all_texts)):
        print("نام پدر معتبر است")
    else:
        print("نام پدر اشتباه وارد شده!")
    if(check(entered_text['شهر تولد:'], all_texts)):
        print("شهر تولد معتبر است")
    else:
        print("شهر تولد اشتباه وارد شده!")

    birth = entered_text['سال تولد:']+'1'+entered_text['ماه تولد:']+'1'+entered_text['روز تولد:']
    birth = convert_numbers.english_to_persian(birth)

    year = convert_numbers.english_to_persian(entered_text['سال تولد:'])
    month = convert_numbers.english_to_persian(entered_text['ماه تولد:'])
    day = convert_numbers.english_to_persian(entered_text['روز تولد:'])
    birth2 = year +'/'+ month +'/'+ day

    sw = 0

    if(check(birth, all_texts)):
        sw = 1
    elif(check(birth2, all_texts)):
        sw = 1

    if(sw):
        print("تاریخ تولد معتبر است")
    else:
        print("تاریخ تولد اشتباه وارد شده!")


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files

# Define text widgets
name = widgets.Text(description='نام:')
last_name = widgets.Text(description='نام خانوادگی:')
birth_day = widgets.Text(description="روز تولد:")
birth_month = widgets.Text(description="ماه تولد:")
birth_year = widgets.Text(description="سال تولد:")
birth_city = widgets.Text(description="شهر تولد:")
father_name = widgets.Text(description="نام پدر:")
mother_name = widgets.Text(description="نام مادر:")

# Create a container to hold the text widgets
text_widgets_container = widgets.VBox([name, last_name, birth_day, birth_month, birth_year, birth_city, father_name, mother_name])

# Upload a photo
photo_upload = None

# Create a button widget
button_widget = widgets.Button(description='اعتبارسنجی')

# Define a callback function for the button click event
def on_button_click(b):
    global photo_upload  # Use the global variable to track photo upload status

    # Validate the entered text
    if not all(widget.value for widget in [name, last_name, birth_day, birth_month, birth_year, birth_city, father_name, mother_name]):
        print("لطفا همه ورودی‌ها را پر کنید!")
        return

    for filename, content in photo_upload.items():
        with open(filename, 'wb') as f:
            f.write(content)
        print(f"{filename} با موفیت آپلود شد")

    # Access the entered text
    entered_text = {widget.description: widget.value for widget in [name, last_name, birth_day, birth_month, birth_year, birth_city, father_name, mother_name]}

    # Print a message
    print("اطلاعات شما دریافت شد. لطفا برای اعتبارسنجی صبر کنید.")

    func(filename, entered_text)

# Attach the callback function to the button click event
button_widget.on_click(on_button_click)

# Display widgets
display(text_widgets_container)

# Enable users to upload a photo
photo_upload = files.upload()

display(button_widget)